In [ ]:
# 손글씨 숫자 인식 연습
'''
이미 학습된 매개변수를 사용하여 학습 과정은 생략하고, 추론 과정만 구현한다.
이 추론 과정을 신경망의 순전파forward propagation라고도 한다.
*기계학습과 마찬가지로, 신경망도 두 단계를 거쳐 학습한다. 1.훈련데이터(학습데이터)를 이용해 가중치 매개변수를 학습하고, 2.추론 단계에서 앞에서 얻은 매개변수를 이용하여 입력데이터를 분류한다.
'''

In [2]:
# MNIST 데이터 셋(손글씨 숫자 이미지 집합) 이용

import sys, os
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

# 최초의 데이터 수집에는 조금 시간이 걸린다.
(x_train, t_train), (x_test, t_test) = \ # (훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블)
    load_mnist(flatten=True, normalize=False) # flatten : 입력 이미지를 1차원 배열로 만들것인지 여부, normalize : 입력 이미지의 픽셀 값을 0.0 ~ 1.0 사이로 정규화 할 것인지 여부
# 사용되지 않았지만, one_hot_lable 옵션은 레이블을 하나의 정답(1,hot)과 나머지 오답(0) 형식으로 만들것인지 여부를 결정한다.

# 각 데이터의 형상 출력
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,)
print(x_test.shape) # (10000, 784)
print(t_test.shape) # (10000,)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import pickle
from dataset.mnist import load_mnist
from PIL import Image # PIL : Python Image Library

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) # numpy로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환한다.
    pil_img.show()

(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label) # 5

print(img.shape) # (784,) 현재 이미지는 1차원 numpy 배열 형태로 저장되어있다.
img = img.reshape(28,28) # 원래 이미지 모양으로 변경 (원래 형상 크기 28*28=784), reshape 메서드를 통해 numpy 배열의 형샹을 바꿀 수 있다.
print(img.shape) # (28,28)

img_show(img)

5
(784,)
(28, 28)


In [12]:
# 추론 데이터 수집 함수
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=False, flatten=True, one_hot_label=False)
    return x_test,t_test

# 네트워크 초기화 함수
def init_network():
    with open("sample_weight.pkl", 'rb') as f: # sample_weight는 학습된 가중치 매개변수가 담겨있는 파일이다. 가중치와 편향 매개변수가 딕셔너리로 저장되어 있다.
        network = pickle.load(f)
        
    return network

##### 활성화 함수 start #####
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

##### 활성화 함수 end #####

# 추론 함수
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [13]:
# 함수 이용해 mnist 분류 신경망 구현

x, t = get_data() # 데이터셋 수집
network = init_network() # 네트워크 생성
accuracy_cnt = 0
for i in range(len(x)): # for문을 돌며 x에 저장된 이미지 데이터를 1장씩 꺼내어 predict() 함수로 분류
    y = predict(network, x[i]) # predict함수는 각 레이블의 확률을 numpy 배열로 반환
    p= np.argmax(y)  # argmax 함수는 이 배열에서 값이 가장 큰(즉, 확률이 가장 높은) 원소의 인덱스를 반환
    if p == t[i]: # 만약 신경망이 예측한 답변(p)와 정답 레이블(t[i])이 같다면(즉, 맞췄다면)
        accuracy_cnt += 1 # 맞힌 숫자를 센다.

print("Accuracy:" + str(float(accuracy_cnt) / len(x))) # (맞힌 숫자 / 전체 이미지)로 정확도를 계산한다.

/Users/seongik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()


Accuracy:0.9207


In [14]:
# load_mnist 함수의 normalize = True옵션은 0~255의 픽셀값을 0.0~1.0 범위로 변환한다.
# 이처럼 데이터를 특정 범위로 변환하는 것을 정규화normalization이라고 하고, 신경망의 입력 데이터에 특정 변환을 가하는 것을 전처리pre-processing라고 한다.

# 현업에서도 신경망(딥러닝)에 전처리를 활발히 사용한다.
# 현업에서는 데이터 전체의 분포를 고려해 전처리 하는 경우가 많다.
# 예를 들어, 데이터 전체 평균과 표준 편차를 이용하여 데이터들이 0을 중심으로 분포하도록 이동하거나, 데이터의 확산 범위를 제약하는 정규화를 수행한다.
# 그 외에도, 전체 데이터를 균일하게 분포시키는 데이터 백색화whitening 등도 있다.

In [15]:
# 입력 데이터와 가중치 매개변수의 '형상'에 주목해보자.

x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape) # (10000,784)
print(x[0].shape) # (784,)
print(W1.shape) # (784,50)
print(W2.shape) # (50,100)
print(W3.shape) # (100,10) 최종적으로는 0~9까지의 10개 숫자로 분류된다.

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [ ]:
# 만약 이미지 100개를 한 묶음으로 만들어 predict 함수에 한번에 넘긴다면 (x의 형상 : 100 * 784)
# 100장 분량의 입력 데이터 결과가 한 번에 출력된다.
# 이처럼 하나로 묶은 입력 데이터를 배치batch라고 한다.

# 배치가 가지는 이점은 다음과 같다.
'''
1. 대부분의 수치 계산 라이브러리가 큰 배열을 효율적으로 처리할 수 있도록 고도로 최적화되어있다.
2. 커다란 신경망에서는 데이터 전송이 병목으로 작용하는 경우가 있는데, 배치처리를 함으로써 버스에 주는 부하를 줄일 수있다.(정확히는 I/O 계산에 들일 시간을 줄이고 CPU 또는 GPU 연산을 하는 시간이 많아진다.)
컴퓨터에서는 큰 배열을 한꺼번에 계산하는 것이 분할된 작은 배열을 여러번 계산하는 것보다 빠르다.

이러한 이유로 배치를 이용했을 때 이미지 1장당 처리 시간을 대폭 줄일 수 있다.
'''

In [22]:
# 짚고 넘어갈 점 1 - argmax의 axis 옵션
# axis=n : n번째 차원을 구성하는 원소에서(n번째 차원을 축으로) 최댓값의 인덱스를 찾도록 한다. 여기서는 1차원을 구성하는 원소들중에서 최댓값을 뽑아낸다.

x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


In [21]:
# 짚고 넘어갈 점 2 - ==연산자의 사용
# numpy 배열끼리 ==를 사용할 시 bool값을 반환한다. 이를 sum에 넣으면 false일 때 count하지 않고(0을 더하고), true일 때 count(1을 더하는)할 수 있다.
y = np.array([1,2,1,0])
t = np.array([1,2,0,0])
print(y==t)
print(np.sum(y==t))

[ True  True False  True]
3


In [17]:
# 배치 처리

x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0,len(x),batch_size): # 배치 크기만큼 건너뛰며 반복한다
    x_batch = x[i:i+batch_size] # 배치 크기만큼의 한 다발을 x_batch로 정한다.
    y_batch = predict(network, x_batch)
    p= np.argmax(y_batch, axis=1)  
    accuracy_cnt += np.sum(p == t[i:i+batch_size]) # 배치단위로 분류한 결과를 실제 답과 비교하여 정확도를 측정한다.
    
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9207


/Users/seongik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()


In [ ]:
# 이번 장에서 배운 내용
# 신경망에서는 활성화 함수로 시그모이드 함수와 ReLU 함수 같은 매끄럽게 변화하는 함수를 이용한다.
# numpy의 다차원 배열을 잘 사용하면 신경망을 효율적으로 구현할 수 있다.
# 기계학습 문제는 크게 회귀와 분류로 나눌 수 있다.
# 출력층의 활성화 함수로는 회귀에서는 주로 항등 함수를, 분류에서는 주로 소프트맥스 함수를 이용한다.
# 분류에서는 출력층의 뉴런 수를 분류하려는 클래스 수와 같게 설정한다.
# 입력 데이터를 묶은 것을 배치라 하며, 추론 처리를 이 배치 단위로 진행하면 결과를 훨씬 빠르게 얻을 수 있다.